# 导入依赖库

In [1]:
# assemble the pretrain model features, to trian a new network 

import mxnet as mx
from mxnet import autograd
from mxnet import gluon
from mxnet import init
from mxnet import nd
from mxnet.gluon import nn
import numpy as np
from tqdm import tqdm
import os

ctx = mx.gpu(2)
batch_size = 128

# 定义网络结构和评估函数

In [2]:
def build_model():
    net = nn.Sequential()
    with net.name_scope():
        net.add(nn.BatchNorm())
        net.add(nn.Dense(4096))
        net.add(nn.BatchNorm())
        net.add(nn.Activation('relu'))
        net.add(nn.Dropout(0.5))
        net.add(nn.Dense(1024))
        net.add(nn.BatchNorm())
        net.add(nn.Activation('relu'))
        net.add(nn.Dropout(0.5))
        net.add(nn.Dense(30))

    net.initialize(ctx=ctx)
    return net

In [3]:
softmax_cross_entropy = gluon.loss.SoftmaxCrossEntropyLoss()

def accuracy(output, labels):
    return nd.mean(nd.argmax(output, axis=1) == labels).asscalar()

def evaluate(net, data_iter):
    loss, acc, n = 0., 0., 0.
    steps = len(data_iter)
    for data, label in data_iter:
        data, label = data.as_in_context(ctx), label.as_in_context(ctx)
        output = net(data)
        acc += accuracy(output, label)
        loss += nd.mean(softmax_cross_entropy(output, label)).asscalar()
    return loss/steps, acc/steps

# 载入DenseNet和Inception-resnet-v2提取的feature

In [4]:
# load train data

feature_densenet = np.load('features/densenet/train_features.npy')
feature_densenet_head = np.load('features/densenet-head/train_features.npy')
feature_inception_res_v2 = np.load('features/inception_res_v2/train_features.npy')

y = nd.load('jd_train_labels.nd')[0]
y = y - 1

features = nd.concat(
                     nd.array(feature_densenet),
                     nd.array(feature_densenet_head),
                     nd.array(feature_inception_res_v2), dim=1)
print(features.shape)

data_iter_train_all = gluon.data.DataLoader(gluon.data.ArrayDataset(features, y), batch_size, shuffle=True)

(17648, 5952)


In [5]:
# load eval data

feature_densenet_eval = np.load('features/densenet/eval_features.npy')
feature_densenet_head_eval = np.load('features/densenet-head/eval_features.npy')
feature_inception_res_v2_eval = np.load('features/inception_res_v2/eval_features.npy')

yy = nd.load('jd_eval_labels.nd')[0]
yy = yy - 1

features_eval = nd.concat(
                          nd.array(feature_densenet_eval),
                          nd.array(feature_densenet_head_eval),
                          nd.array(feature_inception_res_v2_eval), dim=1)
print(features_eval.shape)

data_iter_eval = gluon.data.DataLoader(gluon.data.ArrayDataset(features_eval, yy), 128, shuffle=False)

(600, 5952)


# 把高置信度的测试图片加入训练集（第一次训练时不要使用）

In [26]:
# combin the seleceted test images into train images

features_all = nd.concat(
                     features,
                     nd.array(high_score_test_images), dim=0)
y_all = nd.concat(y, nd.array(high_score_test_image_labels), dim=0)
    
print(features_all.shape)
print(y_all.shape)

data_iter_train_all = gluon.data.DataLoader(gluon.data.ArrayDataset(features_all, y_all), batch_size, shuffle=True)

(20053, 5952)
(20053,)


# 训练模型并保存eval loss最小的最佳模型

In [ ]:
# train the model

net = build_model()

epochs = 30
lr_sch = mx.lr_scheduler.FactorScheduler(step=1000, factor=0.5, stop_factor_lr=1e-8)
trainer = gluon.Trainer(net.collect_params(), 'adam', 
                        {'learning_rate': 4e-4, 'lr_scheduler': lr_sch, 'wd': 0.0})

min_eval_loss = 0.4
for epoch in range(epochs):
    train_loss = 0.
    train_acc = 0.
    steps = len(data_iter_train_all)
    for data, label in data_iter_train_all:
        data, label = data.as_in_context(ctx), label.as_in_context(ctx)
        with autograd.record():
            output = net(data)
            loss = softmax_cross_entropy(output, label)

        loss.backward()
        trainer.step(batch_size)

        train_loss += nd.mean(loss).asscalar()
        train_acc += accuracy(output, label)

    # eval in the eval dataset, keep the model parameters of the mimial eval loss model
    eval_loss, eval_acc = evaluate(net, data_iter_eval)
    if min_eval_loss > eval_loss:
        min_eval_loss = eval_loss
        net.save_params('min_eval_loss_net.params')
    print("Epoch %d. loss: %.4f, acc: %.2f%%, eval_loss: %.4f, eval_acc: %.2f%%" % (epoch+1, train_loss/steps, train_acc/steps*100, eval_loss, eval_acc*100))
    
print("Min eval loss: %.4f" % min_eval_loss)

Epoch 1. loss: 0.1176, acc: 97.57%, eval_loss: 0.3660, eval_acc: 91.59%
Epoch 2. loss: 0.0029, acc: 100.00%, eval_loss: 0.3905, eval_acc: 91.59%


# 载入最佳模型并在验证集上评测

In [20]:
# load the best model
net = build_model()
net.load_params('min_eval_loss_net.params', ctx)  

# eval in the eval dataset
evaluate(net, data_iter_eval)

(0.35200187239825026, 0.92343750000000002)

# 载入测试集

In [8]:
# load test data

feature_densenet_test_b = np.load('features/densenet/test_b_features.npy')
feature_densenet_head_test_b = np.load('features/densenet-head/test_b_features.npy')
feature_inception_res_v2_test_b = np.load('features/inception_res_v2/test_b_features.npy')

features_test_b = nd.concat(
    nd.array(feature_densenet_test_b), 
    nd.array(feature_densenet_head_test_b),
    nd.array(feature_inception_res_v2_test_b), dim=1)
print(features_test_b.shape)

data_iter_test_b = gluon.data.DataLoader(features_test_b, batch_size, shuffle=False)

(3000, 5952)


# 用模型标注测试集，选取置信度高的数据加入训练集重新训练（可迭代多次）

In [9]:
# combine eval and test dataset
features_test_all = nd.concat(features_test_b, features_eval, dim=0)
print(features_test_all.shape)

data_iter_test_all = gluon.data.DataLoader(features_test_all, batch_size, shuffle=False)

(3600, 5952)


In [25]:
high_score_test_images = np.empty((0, features_test_all.shape[1]), float)
high_score_test_image_labels = []
score_thred_hold = 0.99

outputs = np.empty((0,30), float)
for data in data_iter_test_all:
    output = nd.softmax(net(data.as_in_context(ctx))).asnumpy()
    outputs = np.append(outputs, output, axis=0)
    
preds = np.argmax(outputs, axis=1)
scores = np.max(outputs, axis=1)
for i in range(len(outputs)):
    if scores[i] > score_thred_hold:
        high_score_test_images = np.append(high_score_test_images, 
                                           np.expand_dims(features_test_all[i].asnumpy(), axis=0), 
                                           axis=0)
        high_score_test_image_labels.append(preds[i])
        
print("Select %d images" % (len(high_score_test_image_labels)))

Select 2405 images


# 预测测试集并保存结果

In [14]:
outputs = np.empty((0,30), float)
for data in features_test_b:
    output = nd.softmax(net(data.as_in_context(ctx))).asnumpy()
    outputs = np.append(outputs, output, axis=0)

In [15]:
from glob import glob

files = []
for i, file_name in tqdm(enumerate(glob('jd_test_B/*.JPG')), total=3000):
    name = file_name.split('/')[1][:-4]
    files.append(name)

100%|██████████| 3000/3000 [00:00<00:00, 390325.15it/s]


In [16]:
# save the test result

with open('assemble_min_loss.csv', u"w+") as f:
    for i in range(len(outputs)):
        for j in range(30):
            str_row = '%s,%d,%.9f\n' % (files[i], j+1, outputs[i, j])
            f.write(str_row)